In [1]:
# As usual, a bit of setup
import time
from time import process_time
import numpy as np
import matplotlib.pyplot as plt
import LOUPE.WILLOW.loupe as lp
import tensorflow as tf
import h5py
import pandas as pd
import csv
import copy
import math
from utils.data_utils import *
import sys
import re
from utils.spj import Config
from utils.spj import SPJ
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

home_dir = "/home/martnzjulio_a/songze"
#home_dir = "/home/songzeli"

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


# Training Model

In [20]:
def model(all_train, all_val, learning_rate, num_epochs, minibatch_size, print_cost = True):
    """
    Implements a tensorflow neural network: C3D->ATTENTION->CAPTIONING
    
    Arguments:
    H_train -- training set, of shape = [n_train,num_c3d_features,num_proposals]
    Y_train -- caption labels, of shape = [n_train,num_proposals,num_steps+1]
    H_test -- training set, of shape = [n_test,num_c3d_features,num_proposals]
    Y_test -- caption labels, of shape = [n_test,num_proposals,num_steps+1]
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    (H_train, Ipast_train, Ifuture_train, Ycaptions_train, Xcaptions_train) = all_train
    (H_val,   Ipast_val,   Ifuture_val,   Ycaptions_val,   Xcaptions_val)   = all_val
    
    
    # Directory to Save Checkpoint
    checkpoint_dir = home_dir + "/checkpoints/"
    tensorboard_dir =  home_dir + "/tensorboard/"
    
    # Reset Graph
    tf.reset_default_graph()    
    
    # For Consistency
    tf.set_random_seed(1)                             
    seed = 3                                         
    
    # Number of Training Examples
    num_train = H_train.shape[0] 
    num_val = H_val.shape[0] 
    
    # to keep track of costs
    costs = []
    
    # Global Epoch Number
    global_step = tf.Variable(0, name='global_step', trainable=False)
    
    # Model
    config = Config()
    spj = SPJ(config)
        
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer.
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(spj.loss, global_step=global_step)
    
    # Initialize all the variables
    init = tf.global_variables_initializer()
    
    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()
    
    # Tensorboard Loss
    training_summary = tf.summary.scalar("training_loss", spj.loss)
    validation_summary = tf.summary.scalar("validation_loss", spj.loss)
    

    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
    
        # check for latest checkpoint
        latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
        if latest_checkpoint == None:
            # If no check point run the initialization
            print()
            print("No checkpoint exists, initializing parameters...")
            sess.run(init)
        else:
            print()
            print("Restoring from latest checkpoint...")
            saver.restore(sess, latest_checkpoint)
        
        #Tensorboard
        summary_writer = tf.summary.FileWriter(tensorboard_dir,sess.graph)
        
        # Training Loop
        print()
        start = process_time() # Record start time
        for epoch in range(num_epochs):
            
            # Variable to store cost
            epoch_loss = 0.0
            
            # Get minibatches
            num_minibatches = num_train // minibatch_size 
            seed = seed + 1
            minibatches = random_mini_batches(H_train, Ipast_train, Ifuture_train, Ycaptions_train, Xcaptions_train, minibatch_size, seed = 0)

            for counter, minibatch in enumerate(minibatches):
                
                # Select minibatch
                (minibatch_H_train, minibatch_Ipast_train, minibatch_Ifuture_train, minibatch_Ycaptions_train, minibatch_Xcaptions_train) = minibatch
                
                # Run Train Session
                _ , minibatch_loss = sess.run([optimizer, spj.loss], feed_dict={spj._H: minibatch_H_train, spj._Ipast: minibatch_Ipast_train, spj._Ifuture: minibatch_Ifuture_train, spj._x: minibatch_Xcaptions_train, spj._y: minibatch_Ycaptions_train, spj._batchsize: minibatch_size}) #
                #epoch_loss += minibatch_loss / num_minibatches
             
            # Epoch Training Loss
            loss_train, train_summ = sess.run([spj.loss, training_summary], feed_dict={spj._H: H_train, spj._Ipast: Ipast_train, spj._Ifuture: Ifuture_train, spj._x: Xcaptions_train, spj._y: Ycaptions_train, spj._batchsize: num_train }) 
            # Epoch Validation Session
            loss_val, val_summ = sess.run([spj.loss, validation_summary], feed_dict={spj._H: H_val, spj._Ipast: Ipast_val, spj._Ifuture: Ifuture_val, spj._x: Xcaptions_val, spj._y: Ycaptions_val, spj._batchsize: num_val }) #    
            
            # Print cost
            if print_cost == True:
                global_epoch = tf.train.global_step(sess, global_step)//num_minibatches
                print("Epoch: ", global_epoch)
                print ("Training Loss: ", loss_train)
                print ("Validation Loss: ", loss_val)
                summary_writer.add_summary(train_summ, global_step=global_epoch) 
                summary_writer.add_summary(val_summ, global_step=global_epoch) 
            
            # Save Model (every 20 epochs)
            if epoch % 20 == 0:
                print("Saving Checkpoint for global_step " + str(global_epoch))
                saver.save(sess, checkpoint_dir + 'model', global_step = global_epoch)
        
        # Save and Print Processed Time
        end = process_time() 
        print()
        print("Time Elapased: ", end - start)
        
        return 0

# Load Vocab

In [3]:
# Load Vocabulary
embedding_size =512
pad_len = 50
vocabulary,vocab_size = caption_preprocess(home_dir)
emb_matrix,word2id,id2word = get_wordvector(embedding_size,vocab_size,vocabulary)

# Word Embedding Matrix
emb_matrix, word2id, id2word = get_wordvector(embedding_size,vocab_size,vocabulary) #changed by Songze

Total number of words in all captions:  504895
Vocabulary Size (Unique):  13001


# Load Training

In [4]:
# Load Training Data
train_file = home_dir + "/SPJ/train_extreme_small.csv"
train_ids,train_data,train_padded_proposals,train_padded_framestamps = video_preprocess(home_dir, train_file)

# Training Captions
train_padded_sentences,train_padded_sentences_2,train_padded_sentences_id = get_padded_sentences_id(pad_len, train_ids, train_data, word2id) 
Ycaptions_train = np.transpose(copy.deepcopy(train_padded_sentences_2),axes=(0,2,1)).astype(np.int32)
Xcaptions_train = np.transpose(copy.deepcopy(train_padded_sentences),axes=(0,2,1)).astype(np.int32)

# Train Features 
H_train = train_padded_proposals.astype(np.float32)
Ipast_train = temporal_indicator(train_padded_framestamps, mode="past").astype(np.float32)
Ifuture_train = temporal_indicator(train_padded_framestamps, mode="future").astype(np.float32)

# Load Validation

In [5]:
# Load Validation Data
val_file = home_dir + "/SPJ/val_small.csv"
val_ids,val_data,val_padded_proposals,val_padded_framestamps = video_preprocess(home_dir, val_file)

# Validation Captions
val_padded_sentences,val_padded_sentences_2,val_padded_sentences_id = get_padded_sentences_id(pad_len, val_ids, val_data, word2id) 
Ycaptions_val = np.transpose(copy.deepcopy(val_padded_sentences_2),axes=(0,2,1)).astype(np.int32)
Xcaptions_val = np.transpose(copy.deepcopy(val_padded_sentences),axes=(0,2,1)).astype(np.int32)

# Validation Features 
H_val = val_padded_proposals.astype(np.float32)
Ipast_val = temporal_indicator(val_padded_framestamps, mode="past").astype(np.float32)
Ifuture_val = temporal_indicator(val_padded_framestamps, mode="future").astype(np.float32)

print(H_train.shape)

(99, 500, 30)


In [21]:
# Train Model
# learning_rate = 0.001 # first 527 epochs
learning_rate = 0.01
num_epochs = 1000
minibatch_size = 9
all_train = (H_train, Ipast_train, Ifuture_train, Ycaptions_train, Xcaptions_train)
all_val =   (H_val,   Ipast_val,   Ifuture_val,   Ycaptions_val,   Xcaptions_val)
model(all_train, all_val, learning_rate, num_epochs, minibatch_size)


No checkpoint exists, initializing parameters...

Epoch:  1
Training Loss:  117698.86
Validation Loss:  77285.2
Saving Checkpoint for global_step 1
Epoch:  2
Training Loss:  96027.22
Validation Loss:  72378.03
Epoch:  3
Training Loss:  88253.0
Validation Loss:  69943.83
Epoch:  4
Training Loss:  86117.125
Validation Loss:  68482.55
Epoch:  5
Training Loss:  81582.49
Validation Loss:  66605.65
Epoch:  6
Training Loss:  75189.03
Validation Loss:  61707.684
Epoch:  7
Training Loss:  59232.812
Validation Loss:  51116.605
Epoch:  8
Training Loss:  52520.207
Validation Loss:  44460.33
Epoch:  9
Training Loss:  51328.367
Validation Loss:  42847.52
Epoch:  10
Training Loss:  50309.64
Validation Loss:  41775.555
Epoch:  11
Training Loss:  49788.4
Validation Loss:  41381.656
Epoch:  12
Training Loss:  49255.598
Validation Loss:  41261.688
Epoch:  13
Training Loss:  48711.9
Validation Loss:  40958.805
Epoch:  14
Training Loss:  48742.957
Validation Loss:  41041.977
Epoch:  15
Training Loss:  478

KeyboardInterrupt: 